In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../data/dataset/',
                                 transform=T.ToSparseTensor())

In [3]:
torch.cuda.is_available()

True

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [5]:
data = dataset[0]
data.adj_t = data.adj_t.to_symmetric()

In [6]:
data

Data(adj_t=[169343, 169343, nnz=2315598], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [7]:
data = data.to(device)

In [8]:
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

In [9]:
evaluator = Evaluator(name='ogbn-arxiv')

In [10]:
data['x'].shape[1]

128

In [11]:
data['y'][:, 0].unique().shape[0]

40

In [12]:
n_iters = 3000
epochs = 2000
log_steps = 100
input_dim = data['x'].shape[1]
hidden_channels = input_dim * 2
output_dim = data['y'][:, 0].unique().shape[0]
lr_rate = 0.001

In [13]:
class OneLayer(torch.nn.Module):
    def __init__(self, input_dim, hidden_channels, output_dim):
        super(OneLayer, self).__init__()
        
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        
        self.convs.append(SAGEConv(input_dim, hidden_channels))
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        
        self.convs.append(SAGEConv(hidden_channels, output_dim))
        

    def forward(self, x, adj_t):
        x = self.convs[0](x, adj_t)
        x = self.bns[0](x)
        x = self.convs[1](x, adj_t)
        return x.log_softmax(dim=-1)

In [14]:
model = OneLayer(input_dim, hidden_channels, output_dim).to(device)

In [15]:
def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

In [16]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [17]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, data, train_idx, optimizer)
    result = test(model, data, split_idx, evaluator)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 1.0410, Train: 69.13%, Valid: 68.20% Test: 66.73%
Epoch: 200, Loss: 0.9510, Train: 71.13%, Valid: 69.31% Test: 68.35%
Epoch: 300, Loss: 0.9169, Train: 71.88%, Valid: 69.37% Test: 68.88%
Epoch: 400, Loss: 0.9011, Train: 72.36%, Valid: 69.19% Test: 68.70%
Epoch: 500, Loss: 0.8928, Train: 72.58%, Valid: 69.16% Test: 68.54%
Epoch: 600, Loss: 0.8881, Train: 72.67%, Valid: 69.07% Test: 68.34%
Epoch: 700, Loss: 0.8854, Train: 72.69%, Valid: 69.03% Test: 68.16%
Epoch: 800, Loss: 0.8838, Train: 72.74%, Valid: 68.91% Test: 68.16%
Epoch: 900, Loss: 0.8828, Train: 72.77%, Valid: 68.91% Test: 68.13%
Epoch: 1000, Loss: 0.8825, Train: 72.77%, Valid: 68.97% Test: 68.21%
Epoch: 1100, Loss: 0.8817, Train: 72.79%, Valid: 68.93% Test: 68.11%
Epoch: 1200, Loss: 0.8814, Train: 72.86%, Valid: 68.88% Test: 68.02%
Epoch: 1300, Loss: 0.8812, Train: 72.80%, Valid: 68.86% Test: 68.09%
Epoch: 1400, Loss: 0.8815, Train: 72.81%, Valid: 68.81% Test: 68.01%
Epoch: 1500, Loss: 0.8809, Train: 72.85%, V